<a href="https://colab.research.google.com/github/Moore-Lab-UMass/ENCODE-API/blob/main/Pull-DNase-Bams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Jill E. Moore
#Moore Lab - UMass Chan Medical School
#May 2022

import os
import json
import urllib
from urllib import request, parse, error
import requests

In [7]:
def Download_File(name, extension, outputDir): #downloads file to local machine
    if not os.path.exists(outputDir+"/"+name+"."+extension):
        url="https://www.encodeproject.org/files/"+name+"/@@download/"+name+"."+extension
        r = requests.get(url)
        outputFile=open(outputDir+"/"+name+"."+extension, "wb")
        outputFile.write(r.content)
        outputFile.close()

In [5]:
def Extract_Experiment_Metatdata(exp, genome): #extracts experiment metadata
  urlExp = "https://www.encodeproject.org/experiments/"+exp+"/?format=json"
  response = urllib.request.urlopen(urlExp)
  data = json.loads(response.read())

  rfa = data["award"]["rfa"] #project/phase of encode
  lab = data["lab"]["title"] #lab that submitted the experiment
  biosample = data["biosample_ontology"]["term_name"] #short name of biosample

  bam = "NA"
  numReads = "NA"
  replicate = "NA"

  for entry in data["files"]: #loops through files associated with experiment
    if entry["file_type"] == "bam" and entry["status"] == "released" \
      and entry["assembly"] == genome and entry["output_type"] == "alignments":

      bam = entry["accession"] #bam file
      #Download_File(bam, "bam", "/data/output")

      replicate = ",".join([str(i) for i in entry["biological_replicates"]])
      for qc in entry["quality_metrics"]:
        if "mapped" in qc:
          numReads = qc["mapped"]

      for pipeline in entry["analyses"]:
        if pipeline["status"] == "released":
          print(exp + "\t" + bam +"\t" + replicate + "\t" + biosample + "\t" + \
                rfa +"\t"+ lab + "\t" + str(numReads))


In [8]:
## Parameters
genome = "GRCh38"
species = "Homo+sapiens"

## Build query
urlMain = "https://www.encodeproject.org/search/?type=Experiment&status=released" + \
    "&perturbed=false&perturbed=true&assay_title=DNase-seq" + \
    "&replicates.library.biosample.donor.organism.scientific_name=" + species + \
    "&format=json&limit=50" #update limit=all for all datasets

response = urllib.request.urlopen(urlMain)
data = json.loads(response.read())

print("experiment_accession" + "\t" + "bam_accession" + "\t" + "replicate" "\t" + "biosample" + \
        "\t" + "rfa" +"\t"+ "lab" + "\t" + "number_reads")

for entry in data["@graph"]: #loops through experiments
  Extract_Experiment_Metatdata(entry["accession"], genome)

experiment_accession	bam_accession	replicate	biosample	rfa	lab	number_reads
ENCSR799GSS	ENCFF091EIA	1	HCT116	ENCODE4	John Stamatoyannopoulos, UW	101025218
ENCSR163KCQ	ENCFF749NBS	1	HCT116	ENCODE4	John Stamatoyannopoulos, UW	151127322
ENCSR975SCL	ENCFF866CNH	1	HCT116	ENCODE4	John Stamatoyannopoulos, UW	211482144
ENCSR129EWE	ENCFF009SUB	1	HCT116	ENCODE4	John Stamatoyannopoulos, UW	176879358
ENCSR980GWI	ENCFF766VMY	1	HCT116	ENCODE4	John Stamatoyannopoulos, UW	150321310
ENCSR867ITQ	ENCFF801KAM	1	HCT116	ENCODE4	John Stamatoyannopoulos, UW	427815930
ENCSR690ADK	ENCFF523QDP	1	HCT116	ENCODE4	John Stamatoyannopoulos, UW	412241576
ENCSR542UWN	ENCFF625QWI	1	HCT116	ENCODE4	John Stamatoyannopoulos, UW	136754324
ENCSR694YJR	ENCFF213KGB	1	HCT116	ENCODE4	John Stamatoyannopoulos, UW	179509764
ENCSR694YJR	ENCFF191GDK	2	HCT116	ENCODE4	John Stamatoyannopoulos, UW	113278480
ENCSR011VHK	ENCFF528VKG	1	naive B cell	ENCODE4	John Stamatoyannopoulos, UW	107926552
ENCSR794KUS	ENCFF900TQP	1	naive thymus-derived CD